In [1]:
import geopandas as gpd
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Here I am  reading in all of my files after making the clean and it all Tennessee based
res = gpd.read_file('res_refined.geojson')
zipcode = gpd.read_file('zipcode_refined.geojson')
tn = gpd.read_file('tennessee_refined.geojson')
trail = gpd.read_file('trails_refined.geojson')
rail = gpd.read_file('rail_refined.geojson')
park = gpd.read_file('parks_refined.geojson')

### Do lower-income or rural ZIP codes make fewer reservations?

In [3]:
# Here I am just defining the folder I want to store the files in 
exit = 'tableau/'

In [4]:
# Here I am joining the reservation data with the zipcodes and demographics
res_with_zip = gpd.sjoin(res, zipcode[['ZCTA5CE20', 'Median Income', 'Total Population', 'geometry']], how='left', predicate='within')
res_with_zip.head(1)

,parentlocation,park,sitetype,usetype,inventorytype,facilityzip,customerzip,average_totalpaid,sum_nights,average_nights,sum_numberofpeople,average_numberofpeople,geometry,index_right,ZCTA5CE20,Median Income,Total Population
0,Barkley Lake,DOVER,GROUP SHELTER ELECTRIC,Day,CAMPING,310,37015,35.0,0.0,0.0,30.0,30.0,POINT (-87.84306 36.49028),399,37058,56310,7620


In [5]:
# Here I am doing a group by to get the total reservations and other aggregates by zipcode
reservation_counts = res_with_zip.groupby('ZCTA5CE20').agg({'parentlocation': 'count',                  
    'average_totalpaid': 'mean', 'sum_nights': 'sum', 'average_nights': 'mean',
    'sum_numberofpeople': 'sum', 'average_numberofpeople': 'mean'
}).reset_index().rename(columns={'parentlocation': 'reservation_count'})
reservation_counts.head(1)

,ZCTA5CE20,reservation_count,average_totalpaid,sum_nights,average_nights,sum_numberofpeople,average_numberofpeople
0,37013,5,49.0,0.0,0.0,220.0,44.0


In [6]:
# Here I am merging median income and total population back into reservation in preperation for Tableau
zip_analysis = pd.merge(reservation_counts, zipcode[['ZCTA5CE20', 'Median Income', 'Total Population']], on='ZCTA5CE20', how='left')
zip_analysis.to_csv(exit + 'zip_reservation_income_analysis.csv', index=False)
zip_analysis.head(1)

,ZCTA5CE20,reservation_count,average_totalpaid,sum_nights,average_nights,sum_numberofpeople,average_numberofpeople,Median Income,Total Population
0,37013,5,49.0,0.0,0.0,220.0,44.0,61801,102184


### Are there trail-rich areas with few reservations?

In [7]:
# Here I am doing a spatial join for trail and zipcode
trail_with_zip = gpd.sjoin(trail, zipcode[['ZCTA5CE20', 'geometry']], how='left', predicate='intersects')
trail_with_zip.head(1)

,OBJECTID,TSP_UID,TR_NAME,SEGMNT_NAM,SEGTYP,RATING,BLZCLR,BLZMAT,BLZDSC,TRSTAT,...,PRIMARY_NA,SECONDARY_,USE_TYPE,PRIMARY_DE,SECONDAR_1,Shape__Len,GlobalID,geometry,index_right,ZCTA5CE20
0,5,TSP-0079,Connector-3,None,Connector,2.0,None,None,None,Open,...,David Crockett State Park,None,Recreation,State Park,None,1726.712573,e9c7ab6f-359a-4bf6-acec-72b6db5289a1,"LINESTRING (-87.35716 35.25245, -87.35695 35.2...",451.0,38464


In [8]:
# Here I am summing up the length of the trails by zipcode 
trail_by_zip = trail_with_zip.groupby('ZCTA5CE20').agg({'SEGLEN': 'sum'
}).reset_index().rename(columns={'SEGLEN': 'total_trail_length'})
trail_with_zip.head(1)

,OBJECTID,TSP_UID,TR_NAME,SEGMNT_NAM,SEGTYP,RATING,BLZCLR,BLZMAT,BLZDSC,TRSTAT,...,PRIMARY_NA,SECONDARY_,USE_TYPE,PRIMARY_DE,SECONDAR_1,Shape__Len,GlobalID,geometry,index_right,ZCTA5CE20
0,5,TSP-0079,Connector-3,None,Connector,2.0,None,None,None,Open,...,David Crockett State Park,None,Recreation,State Park,None,1726.712573,e9c7ab6f-359a-4bf6-acec-72b6db5289a1,"LINESTRING (-87.35716 35.25245, -87.35695 35.2...",451.0,38464


In [9]:
# Here I am getting the count of reservations by zipcode 
reservation_counts = res_with_zip.groupby('ZCTA5CE20').size().reset_index(name='reservation_count')
reservation_counts.head(1)

,ZCTA5CE20,reservation_count
0,37013,5


In [10]:
# Here I am merging trails, zipcodes and reservations to prepare for Tableau
trail_res_combo = pd.merge(trail_by_zip, reservation_counts, on='ZCTA5CE20', how='outer').fillna(0)
trail_res_combo = pd.merge(trail_res_combo, zipcode[['ZCTA5CE20', 'Median Income', 'Total Population']], on='ZCTA5CE20', how='left')
trail_res_combo.to_csv(exit + 'trail_vs_reservations.csv', index=False)
trail_res_combo.head(1)

,ZCTA5CE20,total_trail_length,reservation_count,Median Income,Total Population
0,37013,0.180698,5.0,61801,102184


### What is the average number of people per reservation, and does that correlate with proximity to parks/trails?

In [11]:
# Here I am adding a buffer to the trails and reservations to look at the affects of proximity to 
# trails on the number of reservations placed
trail_buffer = trail
trail_buffer = trail_buffer.to_crs(epsg=3857)
trail_buffer['geometry'] = trail_buffer.buffer(500)
trail_buffer = trail_buffer.to_crs('EPSG:4326')
res_near_trail = gpd.sjoin(res, trail_buffer[['geometry']], how='left', predicate='within')
res_near_trail = res_near_trail.reset_index()
res['near_trail'] = res.index.isin(res_near_trail['index'])

In [12]:
# Here I am doing the same thing for parks and reservations
park_buffer = park
park_buffer = park_buffer.to_crs(epsg=3857)
park_buffer['geometry'] = park_buffer.buffer(250)
park_buffer = park_buffer.to_crs('EPSG:4326')
res_near_park = gpd.sjoin(res, park_buffer[['geometry']], how='left', predicate='within')
res['near_park'] = res_near_park.index_right.notnull()

In [13]:
# Now that everything is combined I am doing  group by to prepare the data for visualizing in Tableau
group_stats = res.groupby(['near_trail', 'near_park']).agg({
    'average_numberofpeople': ['mean', 'count']}).reset_index()
group_stats.columns = ['near_trail', 'near_park', 'avg_people', 'num_reservations']
group_stats.to_csv(exit + 'group_size_vs_proximity.csv', index=False)

### Which ZIP codes generate the most outdoor reservations?

In [14]:
# Here I am looking at the count of origin zips to show where people are coming from
origin_zip_counts = res.groupby('customerzip').size().reset_index(name='reservation_count')
# I also wanted to add in some of the other stats
origin_zip_stats = res.groupby('customerzip').agg({'average_totalpaid': 'mean',
    'average_numberofpeople': 'mean', 'sum_nights': 'sum'}).reset_index()
# Merge the two together for an origin summary and export for Tableau
zip_origin_summary = pd.merge(origin_zip_counts, origin_zip_stats, on='customerzip', how='left')
zip_origin_summary.to_csv(exit + 'zip_origin_reservations.csv', index=False)

### Can we rank ZIPs for trail expansion priority?

In [15]:
# I am joining trail with zip to see how much trail is in each of the zipcodes
trail_zip = gpd.sjoin(trail, zipcode[['ZCTA5CE20', 'geometry']], how='left', predicate='intersects')
trail_by_zip = trail_zip.groupby('ZCTA5CE20')['SEGLEN'].sum().reset_index().rename(columns={'SEGLEN': 'trail_mileage'})
trail_by_zip.head(1)

,ZCTA5CE20,trail_mileage
0,37013,0.180698


In [16]:
# Here I am joining reservations with zip to find where there are lots of reservations made
res_zip = gpd.sjoin(res, zipcode[['ZCTA5CE20', 'geometry']], how='left', predicate='within')
res_by_zip = res_zip.groupby('ZCTA5CE20').size().reset_index(name='reservation_count')
res_by_zip.head(1)

,ZCTA5CE20,reservation_count
0,37013,5


In [17]:
# Here I am creating a summary by zipcode for analysis
zip_stats = zipcode[['ZCTA5CE20', 'Median Income', 'Total Population']]
zip_summary = res_by_zip.merge(trail_by_zip, on='ZCTA5CE20', how='outer').merge(zip_stats, on='ZCTA5CE20', how='left').fillna(0)

In [18]:
# Here I am scaling my value for easier calculations
scaler = MinMaxScaler()
zip_summary[['norm_trail', 'norm_res', 'norm_income', 'norm_pop']] = scaler.fit_transform(
    zip_summary[['trail_mileage', 'reservation_count', 'Median Income', 'Total Population']]
)

In [19]:
# We have a few calculations that are opposite from the rest, like trail being larger means that 
# there is less need for trails. So we are going to invert these ones
zip_summary['trail_need'] = 1 - zip_summary['norm_trail']
zip_summary['income_priority'] = 1 - zip_summary['norm_income']

In [20]:
# Here we are weighting the scores, so we can prioritize where reservation are hot and 
# where there are few trails
zip_summary['priority_score'] = (zip_summary['trail_need'] * 0.4 + zip_summary['norm_res'] * 0.3 +
    zip_summary['income_priority'] * 0.2 + zip_summary['norm_pop'] * 0.1)

In [21]:
# Here we are just ranking zipcodes by the priority score we just made to see where trails are most
# needed
zip_summary['trail_priority_rank'] = zip_summary['priority_score'].rank(ascending=False)

In [22]:
# Here we are exporting for Tableau
zip_summary.to_csv(exit + 'trail_expansion_priority.csv', index=False)
zip_summary.head(1)

,ZCTA5CE20,reservation_count,trail_mileage,Median Income,Total Population,norm_trail,norm_res,norm_income,norm_pop,trail_need,income_priority,priority_score,trail_priority_rank
0,37013,5.0,0.180698,61801,102184,0.001806,0.022523,0.999886,1.0,0.998194,0.000114,0.506057,6.0


In [23]:
# Here I am adding in some geometry and creating a Geo file for Taleau
zip_geo = zipcode[['ZCTA5CE20', 'geometry']]
zip_summary = zip_summary.merge(zip_geo, on = 'ZCTA5CE20', how = 'inner')
zip_gdf = gpd.GeoDataFrame(zip_summary, geometry='geometry')
zip_gdf.set_crs("EPSG:4326", inplace=True)
zip_gdf.to_file(exit + "trail_expansion_priority.geojson", driver="GeoJSON")

### How does Income vary across ZIPs with high vs. low recreation access scores?

In [36]:
# Here we will merge trail and reservation data to zipcode and fill in na with 0's
zip = zipcode[['ZCTA5CE20', 'Total Population', 'Median Income]]
access = zip.merge(trail_by_zip, on='ZCTA5CE20', how='left')
access = access.merge(res_by_zip, on='ZCTA5CE20', how='left')
access = access.fillna(0)

In [37]:
# Here we are making a flag for high and low access based on trail mileage
trail_mean = access['trail_mileage'].mean()
access['access_group'] = access['trail_mileage'].apply(
    lambda x: 'High Access' if x >= trail_mean else 'Low Access')

In [38]:
# Here we will add 
access.to_csv(exit + 'simple_access_income.csv', index=False)